In [ ]:
# load libraries and scripts
using GLM
using DataFrames
using Statistics
using Plots.PlotMeasures

include("../scripts/tools.jl")
include("../scripts/get_grid.jl")
include("../scripts/get_params.jl")
include("../scripts/stable.jl")
include("../scripts/dynamics.jl")
include("../scripts/disturbances.jl")
include("../scripts/plotting_julian.jl")
include("../scripts/ps_analysis.jl");

In [ ]:
#load borders
border, scale_factor = import_border("../data/borders/border.json")

dx = 0.005
grid = get_grid(border, dx);

In [ ]:
#scatter(grid.coord[grid.isgrid,2],grid.coord[grid.isgrid,1])
#scatter!(grid.coord[grid.isborder,2],grid.coord[grid.isborder,1])

In [ ]:
## Sweep paramters and calculate r^2 between th_discrete and th_continuos
tmin = 1e-7
tmax = 1e-5
step = 2e-7
Nstep = Int64(floor((tmax-tmin)/step)+1)
vals = zeros(Nstep, 2)
i = 1
for j=tmin:step:tmax
    println(j)
    contmod = get_params(grid, scale_factor, "../data/disc/pantagruel.h5", Niter=10,
        dmax = 2*dx, patch=1900.0, bmin=56.0, tau=j);
    compute_stable_sol!(contmod, interval = 10000, Niter = 60000, tol = 1e-7);
    th_disc = get_discrete_values(contmod.coord[contmod.isgrid,:], dm.coord, contmod.th[contmod.isgrid]);
    # df = DataFrame(A=dm.th, B=th_disc)
    # lin = lm(@formula(A  ~ B), df)
    # println(coef(lin))
    # println(r2(lin))
    vals[i,1] = j
    vals[i,2] = 1 - sum((dm.th .- th_disc).^2)/sum((th_disc .- mean(th_disc)).^2)
    i += 1
end

In [ ]:
## Sweep paramters and calculate r^2 between th_discrete and th_continuos
tmin = 0.0
tmax = 100.0
step = 2.0
Nstep = Int64(floor((tmax-tmin)/step)+1)
vals = zeros(Nstep, 2)
i = 1
for j=tmin:step:tmax
    println(j)
    contmod = get_params(grid, scale_factor, "../data/disc/pantagruel.h5", Niter=10,
        dmax = 2*dx, patch=1900.0, bmin=j, tau=5e-7);
    compute_stable_sol!(contmod, interval = 10000, Niter = 120000, tol = 1e-7);
    th_disc = get_discrete_values(contmod.coord[contmod.isgrid,:], dm.coord, contmod.th[contmod.isgrid]);
    # df = DataFrame(A=dm.th, B=th_disc)
    # lin = lm(@formula(B  ~ A), df)
    # println(coef(lin))
    # println(r2(lin))
    vals[i,1] = j
    vals[i,2] = 1 - sum((dm.th .- th_disc).^2)/sum((th_disc .- mean(th_disc)).^2)
    i += 1
end

In [ ]:
## Sweep paramters and calculate r^2 between th_discrete and th_continuos
tmin = 1800.0
tmax = 2200.0
step = 20.0
Nstep = Int64(floor((tmax-tmin)/step)+1)
vals = zeros(Nstep, 2)
i = 1
for j=tmin:step:tmax
    println(j)
    contmod = get_params(grid, scale_factor, "../data/disc/pantagruel.h5", Niter=10,
        dmax = 2*dx, patch=j, bmin=56.0, tau=1e-8);
    compute_stable_sol!(contmod, interval = 10000, Niter = 60000, tol = 1e-7);
    th_disc = get_discrete_values(contmod.coord[contmod.isgrid,:], dm.coord, contmod.th[contmod.isgrid]);
    # df = DataFrame(A=dm.th, B=th_disc)
    # lin = lm(@formula(B  ~ A), df)
    # println(coef(lin))
    # println(r2(lin))
    vals[i,1] = j
    vals[i,2] = 1 - sum((dm.th .- th_disc).^2)/sum((th_disc .- mean(th_disc)).^2)
    i += 1
end

In [ ]:
## Sweep paramters and calculate r^2 between th_discrete and th_continuos
tmin = 5
tmax = 100
step = 2
Nstep = Int64(floor((tmax-tmin)/step)+1)
vals = zeros(Nstep, 2)
i = 1
for j=tmin:step:tmax
    println(j)
    contmod = get_params(grid, scale_factor, "../data/disc/pantagruel.h5", Niter=j,
        dmax = 2*dx, patch=1900.0, bmin=56.0, tau=5e-7);
    compute_stable_sol!(contmod, interval = 10000, Niter = 60000, tol = 1e-7);
    th_disc = get_discrete_values(contmod.coord[contmod.isgrid,:], dm.coord, contmod.th[contmod.isgrid]);
    # df = DataFrame(A=dm.th, B=th_disc)
    # lin = lm(@formula(A  ~ B), df)
    # println(coef(lin))
    # println(r2(lin))
    vals[i,1] = j
    vals[i,2] = 1 - sum((dm.th .- th_disc).^2)/sum((th_disc .- mean(th_disc)).^2)
    i += 1
end

In [ ]:
vals[argmax(vals[:,2])]

In [ ]:
plot(vals[:,1], vals[:,2])

In [ ]:
# compute or load grid parameters
contmod = get_params(grid, scale_factor, "../data/disc/pantagruel.h5", Niter=10,
    dmax = 2*dx, patch=1900.0, bmin=52.0, tau=5.0e-7);

In [ ]:
compute_stable_sol!(contmod, interval = 10000, Niter = 60000, tol = 1e-7);
println(maximum(contmod.th) - minimum(contmod.th))
hm_plot(contmod, contmod.th, cb_title = "\$\\theta(x)\$", borders=[border])

In [ ]:
th_disc = get_discrete_values(contmod.coord[contmod.isgrid,:], dm.coord, contmod.th[contmod.isgrid]);
df = DataFrame(A=dm.th, B=th_disc)
lin = lm(@formula(B  ~ A), df)
println(coef(lin))
println(r2(lin))
b=coef(lin)[1]
m=coef(lin)[2]
scatter(dm.th, th_disc)
plot!([minimum(dm.th);maximum(dm.th)],[b+m*minimum(dm.th);-b+m*maximum(dm.th)],color=:red)

In [ ]:
r = 1 - sum((dm.th .- th_disc).^2)/sum((th_disc .- mean(th_disc)).^2)

In [ ]:
dm = load_discrete_model("../data/disc/pantagruel.h5", scale_factor)
#th = set_ref_phase!(contmod, vec(dm.coord[1,:]), th_ref = 0.0)
th_disc = get_discrete_values(contmod.coord[contmod.isgrid,:], dm.coord, contmod.th[contmod.isgrid]);

In [ ]:
scatter(dm.th, th_disc)
th_min = minimum(dm.th)
th_max = maximum(dm.th)
plot!([th_min; th_max], [th_min; th_max], xlabel="\$\\theta_i^{\\rm ref}\$",
    ylabel="\$\\theta_i\$",color=:red, lw = 2, legend=false, size=(400,400))
# savefig("../numerics/theta_ref_vs_cont.pdf")

In [ ]:
clim = extrema([dm.th; th_disc])
p1 = disc_plot(dm.coord, dm.th, borders=[border], clim=clim, c=:viridis, cbar=false, cb_title="\$\\theta_i\$")
p2 = hm_plot(contmod, contmod.th, cbar=false, borders=[border], c=:viridis, clim=clim)
p3 = scatter([0],[0],zcolor=[NaN],clim=clim, c=:viridis, xlims=(1,1.1), framestyle=:none, label="",
cbar_title="\$\\theta\$")
l = @layout [a b c{0.035w}]
p_all = plot(p1, p2,  p3, layout=l, link=:all, title=["Discrete" "Continuous" ""], size=(1200,400))
savefig("../numerics/discontth0.pdf")

In [ ]:
p1 = scatter(dm.th, th_disc)
th_min = minimum(dm.th)
th_max = maximum(dm.th)
plot!([th_min; th_max], [th_min; th_max], xlabel="\$\\theta_i^{\\rm ref}\$",
    ylabel="\$\\theta_i\$",color=:red, lw = 2, legend=false, left_margin=5mm, bottom_margin=8mm)
p2 = disc_plot(dm.coord, dm.th .- th_disc, borders=[border], c=:viridis, cb_title="\$\\theta_{i}^{\\rm ref}-\\theta_i\$")
l = @layout [a{0.4w} b]
plot(p1, p2, layout=l, size=(1200,400))
savefig("../numerics/comparison.pdf")

In [ ]:
add_local_disturbance!(contmod, [741.57038805;-1078.62366301]/scale_factor, -9.0, 0.01, dx);

In [ ]:
ts, thetas, omegas = perform_dyn_sim(contmod, interval=1,
    Ndt=5000, dt=0.01, method="crank-nicolson");

In [ ]:
#hm_movie(contmod, ts, omegas, tstart = 0.0, tend = 20.0, interval = 1)
c = [500 -250; 0.0 0.0; -500 500; 500. 500.; -1500.0 -1000.0; 750.0 -1000; -1000 -400] / scale_factor
time_plot(contmod, ts, omegas, c, borders = [border], tstart = 0.0, tend = 25.0)